In [152]:
import torch
import torch.nn as nn

import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
%matplotlib inline

In [153]:
with open('./PITdf.pkl','rb') as f:
    PITdf = pickle.load(f)
with open('./GREdf.pkl','rb') as f:
    GREdf = pickle.load(f)
with open('./JONdf.pkl','rb') as f:
    JONdf = pickle.load(f)
with open('./MGTdf.pkl','rb') as f:
    MGTdf = pickle.load(f)
with open('./WASdf.pkl','rb') as f:
    WASdf = pickle.load(f)
with open('./WHLdf.pkl','rb') as f:
    WHLdf = pickle.load(f)
with open('./PKSdf.pkl','rb') as f:
    PKSdf = pickle.load(f)
with open('./CBGdf.pkl','rb') as f:
    CBGdf = pickle.load(f)
with open('./STUdf.pkl','rb') as f:
    STUdf = pickle.load(f)
with open('./NPHdf.pkl','rb') as f:
    NPHdf = pickle.load(f)
with open('./ELVdf.pkl','rb') as f:
    ELVdf = pickle.load(f)
with open('./YGTdf.pkl','rb') as f:
    YGTdf = pickle.load(f)
with open('./NCSdf.pkl','rb') as f:
    NCSdf = pickle.load(f)
with open('./BUTdf.pkl','rb') as f:
    BUTdf = pickle.load(f)
with open('./KITdf.pkl','rb') as f:
    KITdf = pickle.load(f)

In [154]:
print(PITdf.head())

   Year  Month  Day  Hour  Minute  DNI  Wind Speed  Wind Direction    wind_x  \
0  1998      1    1     0       0  0.0         0.6           241.0 -0.524772   
1  1998      1    1     0      30  0.0         0.6           241.0 -0.524772   
2  1998      1    1     1       0  0.0         0.6           238.0 -0.508829   
3  1998      1    1     1      30  0.0         0.6           238.0 -0.508829   
4  1998      1    1     2       0  0.0         0.6           225.8 -0.430146   

     wind_y     day_x     day_y    time_x    time_y  max_possible_DNI  \
0 -0.290886  0.017213  0.999852  0.000000  1.000000               0.0   
1 -0.290886  0.017213  0.999852  0.130526  0.991445               0.0   
2 -0.317952  0.017213  0.999852  0.258819  0.965926               0.0   
3 -0.317952  0.017213  0.999852  0.382683  0.923880               0.0   
4 -0.418299  0.017213  0.999852  0.500000  0.866025               0.0   

   cloudiness_factor  
0           0.773816  
1           0.020232  
2          

In this attempt, I will use only the DNI from PGH plus the 6 closest cities (PIT, BUT, GRE, WAS, WHL, STU, ELV). The shape of this data will have each column be the DNI from a different city. The rows will be measurement instances.

In [155]:
all_data = np.concatenate((PITdf['DNI'].values.reshape(-1,1), BUTdf['DNI'].values.reshape(-1,1), \
                           GREdf['DNI'].values.reshape(-1,1), ELVdf['DNI'].values.reshape(-1,1), \
                           WHLdf['DNI'].values.reshape(-1,1), STUdf['DNI'].values.reshape(-1,1), ),axis = 1)


For training purposes, I will use 5 years of data for training and 1 for testing. Training data will be 1998-2002 and testing data will be 2003.

In [156]:
test_length = 365 * 48 * 1
train_length = 365 * 48 * 5
train_data = all_data[:train_length,:]
test_data = all_data[train_length:train_length + test_length, :]

I will normalize the data between -1 and 1

In [157]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range=(-1, 1))
train_data_normalized = scaler.fit_transform(train_data)
train_data_normalized




array([[-1., -1., -1., -1., -1., -1.],
       [-1., -1., -1., -1., -1., -1.],
       [-1., -1., -1., -1., -1., -1.],
       ...,
       [-1., -1., -1., -1., -1., -1.],
       [-1., -1., -1., -1., -1., -1.],
       [-1., -1., -1., -1., -1., -1.]])

Convert np.array to tensor

In [158]:
train_data_normalized = torch.FloatTensor(train_data_normalized)
test_data = torch.FloatTensor(test_data)

I will use the previous 1 time step information from all locations to predict the DNI in PIT at timestep t

In [159]:
t_steps_input = 1
train_data_normalized_input = train_data_normalized[:-t_steps_input,:]
train_data_normalized_targets = train_data_normalized[t_steps_input:,1]

Now I will attempt to build a class defining a FFNN

In [160]:
class Feedforward(torch.nn.Module):
    def __init__(self, input_size, hidden_size):
        super(Feedforward, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.fc1 = torch.nn.Linear(self.input_size, self.hidden_size)
        self.relu = torch.nn.ReLU()
        self.fc2 = torch.nn.Linear(self.hidden_size, 1)
        self.sigmoid = torch.nn.Sigmoid()
        
    def forward(self, x):
        hidden = self.fc1(x)
        relu = self.relu(hidden)
        output = self.fc2(relu)
        output = self.sigmoid(output)
        return output


In [161]:
model = Feedforward(6,100)
criterion = torch.nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr = 0.01)


In [162]:
model.eval()
y_pred = model(train_data_normalized_input)
before_train = criterion(y_pred.squeeze(), train_data_normalized_targets)
print('Test loss before training', before_train.item())

Test loss before training 1.556161880493164


In [163]:
model.train()
epoch = 100

for epoch in range(epoch):
    
    optimizer.zero_grad()
    
    y_pred = model(train_data_normalized_input)
    loss = criterion(y_pred.squeeze(), train_data_normalized_targets)
    if epoch%10 == 0:
        print('Epoch {}: train loss: {}'.format(epoch, loss.item()))
    
    loss.backward()
    optimizer.step()

Epoch 0: train loss: 1.556161880493164
Epoch 10: train loss: 1.1964085102081299
Epoch 20: train loss: 1.0244988203048706
Epoch 30: train loss: 0.9374988079071045
Epoch 40: train loss: 0.8876590728759766
Epoch 50: train loss: 0.8560982942581177
Epoch 60: train loss: 0.8345780372619629
Epoch 70: train loss: 0.8190730214118958
Epoch 80: train loss: 0.8074215650558472
Epoch 90: train loss: 0.7983716726303101
Epoch 100: train loss: 0.7911536693572998
Epoch 110: train loss: 0.7852700352668762
Epoch 120: train loss: 0.7803866267204285
Epoch 130: train loss: 0.7762705683708191
Epoch 140: train loss: 0.7727553248405457
Epoch 150: train loss: 0.7697189450263977
Epoch 160: train loss: 0.7670698165893555
Epoch 170: train loss: 0.764738142490387
Epoch 180: train loss: 0.7626698017120361
Epoch 190: train loss: 0.7608221173286438
Epoch 200: train loss: 0.7591611742973328
Epoch 210: train loss: 0.7576594948768616
Epoch 220: train loss: 0.7562947273254395
Epoch 230: train loss: 0.7550485730171204
Epoch